In [1]:
import tensorflow as tf
physical_devices = tf.config.experimental.list_physical_devices('GPU')
if len(physical_devices) > 0:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)

import core.utils as utils
from core.yolov4 import filter_boxes
#from tensorflow.python.keras.models import load_model
from PIL import Image
import cv2
import numpy as np

from tensorflow.python.saved_model import tag_constants


In [2]:
weights = 'models/yolov4-default/yolov4.weights' #path to weights file
classes_file = 'models/yolov4-default/classes.txt'
model_path1 = 'models/yolov4-default/yolov4.h5'
model_path2 = 'checkpoints/yolov4'

input_size = 416 #define input size of export model
score_thres = 0.2 #define score threshold'

image = './data/kite.jpg' #path to input image
output = 'result.png' #path to output image
iou = 0.45 #iou threshold
score = 0.25 #score threshold

In [3]:
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.compat.v1.InteractiveSession(config=config)

saved_model_loaded = tf.saved_model.load(model_path2, tags=[tag_constants.SERVING])
infer = saved_model_loaded.signatures['serving_default']

In [4]:
original_image = cv2.imread(image)
original_image = cv2.cvtColor(original_image, cv2.COLOR_BGR2RGB)

# image_data = utils.image_preprocess(np.copy(original_image), [input_size, input_size])
image_data = cv2.resize(original_image, (input_size, input_size))
image_data = image_data / 255.
# image_data = image_data[np.newaxis, ...].astype(np.float32)

images_data = []
for i in range(1):
    images_data.append(image_data)
images_data = np.asarray(images_data).astype(np.float32)

batch_data = tf.constant(images_data)
pred_bbox = infer(batch_data)
for key, value in pred_bbox.items():
    boxes = value[:, :, 0:4]
    pred_conf = value[:, :, 4:]

boxes, scores, classes, valid_detections = tf.image.combined_non_max_suppression(
    boxes=tf.reshape(boxes, (tf.shape(boxes)[0], -1, 1, 4)),
    scores=tf.reshape(
        pred_conf, (tf.shape(pred_conf)[0], -1, tf.shape(pred_conf)[-1])),
    max_output_size_per_class=50,
    max_total_size=50,
    iou_threshold=iou,
    score_threshold=score
)
pred_bbox = [boxes.numpy(), scores.numpy(), classes.numpy(), valid_detections.numpy()]

image = utils.draw_bbox(original_image, pred_bbox, classes_file)
image = Image.fromarray(image.astype(np.uint8))
image.show()
#image = cv2.cvtColor(np.array(image), cv2.COLOR_BGR2RGB)
#cv2.imwrite(output, image)


error: OpenCV(4.6.0) :-1: error: (-5:Bad argument) in function 'rectangle'
> Overload resolution failed:
>  - Can't parse 'pt1'. Sequence item with index 0 has a wrong type
>  - Can't parse 'pt1'. Sequence item with index 0 has a wrong type
>  - Can't parse 'rec'. Expected sequence length 4, got 2
>  - Can't parse 'rec'. Expected sequence length 4, got 2
